In [13]:
from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image
import torch
import requests

In [14]:
image = Image.open('data/blend.png')

In [15]:
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")

In [ ]:
inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [6]:
logits = outputs.logits
bboxes = outputs.pred_boxes

In [7]:
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.1, target_sizes=target_sizes)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

Detected frisbee with confidence 0.15 at location [486.1, 401.41, 712.35, 588.89]
Detected skis with confidence 0.134 at location [371.52, 44.93, 461.61, 371.32]
Detected tv with confidence 0.175 at location [230.56, 0.37, 445.96, 72.41]
Detected frisbee with confidence 0.351 at location [513.18, 185.19, 709.96, 311.59]
Detected scissors with confidence 0.109 at location [838.38, 102.87, 985.34, 213.84]


In [8]:
from dotenv import load_dotenv
from roboflow import Roboflow
import os
_ = load_dotenv()

In [12]:

from roboflow import Roboflow
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace("vex2").project("detect_rings")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.58, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to detect_rings-1 in yolov8:: 100%|██████████| 72/72 [00:00<00:00, 14093.89it/s]


In [13]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

100%|██████████| 6.25M/6.25M [00:00<00:00, 42.0MB/s]


In [19]:
results = model.train(data="/home/sgarg26/Documents/p5/yolo/detect_rings/data.yaml", epochs=100)

New https://pypi.org/project/ultralytics/8.2.59 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/sgarg26/Documents/p5/yolo/detect_rings/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256,

train: Scanning /home/sgarg26/Documents/p5/yolo/detect_rings/train/labels... 27 images, 0 backgrounds, 0 corrupt: 100%|██████████| 27/27 [00:00<00:00, 1302.23it/s]

train: New cache created: /home/sgarg26/Documents/p5/yolo/detect_rings/train/labels.cache



val: Scanning /home/sgarg26/Documents/p5/yolo/detect_rings/valid/labels... 2 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:00<00:00, 2801.81it/s]

...

    100/100       2.1G     0.3919     0.8944      1.179         11        640: 100%|██████████| 2/2 [00:00<00:00,  8.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 27.61it/s]

                   all          2          2      0.983          1      0.995      0.895



100 epochs completed in 0.037 hours.
Optimizer stripped from /home/sgarg26/Documents/p5/runs/detect/train3/weights/last.pt, 6.3MB
Optimizer stripped from /home/sgarg26/Documents/p5/runs/detect/train3/weights/best.pt, 6.3MB

Validating /home/sgarg26/Documents/p5/runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 47.64it/s]


                   all          2          2      0.982          1      0.995      0.995
Speed: 0.3ms preprocess, 4.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /home/sgarg26/Documents/p5/runs/detect/train3


In [20]:
output = model("data/vex.jpg")


image 1/1 /home/sgarg26/Documents/p5/yolo/data/vex.jpg: 384x640 2 ringss, 73.1ms
Speed: 1.4ms preprocess, 73.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


In [33]:
output[0].tojson

<bound method Results.tojson of ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'rings'}
obb: None
orig_img: array([[[  0,   1,   4],
        [ 90,  95,  98],
        [146, 151, 154],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   3],
        [ 94,  99, 102],
        [118, 123, 126],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   3],
        [101, 106, 109],
        [ 99, 104, 107],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  1,   2,   0],
        [130, 131, 129],
        [129, 130, 128],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  1,   2,   0],
        [130, 131, 129],
        [129, 130, 128],
        ...,
        [  0,   0,   0],
       

In [35]:
model.predict("data/vex.jpg", save=True)


image 1/1 /home/sgarg26/Documents/p5/yolo/data/vex.jpg: 384x640 2 ringss, 17.9ms
Speed: 2.6ms preprocess, 17.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /home/sgarg26/Documents/p5/runs/detect/train32


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'rings'}
 obb: None
 orig_img: array([[[  0,   1,   4],
         [ 90,  95,  98],
         [146, 151, 154],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   3],
         [ 94,  99, 102],
         [118, 123, 126],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   3],
         [101, 106, 109],
         [ 99, 104, 107],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        ...,
 
        [[  1,   2,   0],
         [130, 131, 129],
         [129, 130, 128],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  1,   2,   0],
         [130, 131, 129],
         [129, 130, 128],
         ...,
         [  0,   0,

In [12]:
from ultralytics import YOLO

model = YOLO("best.pt")

model.predict("data/vex.jpg", save=True)


image 1/1 /home/sgarg26/Documents/p5/yolo/data/vex.jpg: 384x640 2 ringss, 71.3ms
Speed: 1.5ms preprocess, 71.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /home/sgarg26/Documents/p5/runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'rings'}
 obb: None
 orig_img: array([[[  0,   1,   4],
         [ 90,  95,  98],
         [146, 151, 154],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   3],
         [ 94,  99, 102],
         [118, 123, 126],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   3],
         [101, 106, 109],
         [ 99, 104, 107],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        ...,
 
        [[  1,   2,   0],
         [130, 131, 129],
         [129, 130, 128],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  1,   2,   0],
         [130, 131, 129],
         [129, 130, 128],
         ...,
         [  0,   0,